# Trains

In [6]:
import pandas as pd
import json
import os

# Nom du fichier que vous venez de créer
fichier_json = 'gares-de-voyageurs.geojson'

# Vérification que le fichier existe
if not os.path.exists(fichier_json):
    print(f"❌ Erreur : Le fichier '{fichier_json}' est introuvable.")
    print("Assurez-vous d'avoir créé le fichier et collé les données dedans.")
else:
    # 1. Chargement des données depuis le fichier
    with open(fichier_json, 'r', encoding='utf-8') as f:
        data = json.load(f)

    # 2. Transformation en tableau Pandas
    # On cible la clé 'features' qui contient la liste des gares
    df = pd.json_normalize(data['features'])

    # 3. Nettoyage des colonnes
    # Les colonnes s'appellent "properties.nom", "geometry.coordinates", etc.
    # On va simplifier les noms
    df.columns = [c.replace('properties.', '').replace('geometry.', '') for c in df.columns]

    # 4. Séparation de la latitude et longitude
    # La colonne 'coordinates' est une liste [Lon, Lat]. On la divise en deux.
    # Attention : GeoJSON met la Longitude en premier !
    df['longitude'] = df['coordinates'].apply(lambda x: x[0] if isinstance(x, list) else None)
    df['latitude'] = df['coordinates'].apply(lambda x: x[1] if isinstance(x, list) else None)

    # 5. Sélection et renommage final
    # On garde ce qui vous intéresse pour votre modèle immo
    colonnes_utiles = [
        'nom', 
        'codeinsee', 
        'latitude', 
        'longitude', 
        'segment_drg' # Info utile: A = Grosse gare TGV, C = Petite gare
    ]
    
    # On filtre pour ne garder que les colonnes existantes
    colonnes_finales = [c for c in colonnes_utiles if c in df.columns]
    df_clean = df[colonnes_finales]

    print(f"✅ Succès ! {len(df_clean)} gares récupérées.")
    print(df_clean.head())

    # 6. Export en CSV pour utilisation future
    df_clean.to_csv("gares_sncf_final.csv", index=False)
    print("📁 Fichier 'gares_sncf_final.csv' créé.")

✅ Succès ! 2785 gares récupérées.
                     nom codeinsee   latitude  longitude segment_drg
0              Abancourt     60001  49.685224   1.774306           C
1               Abbaretz     44001  47.554643  -1.524416           C
2              Abbeville     80001  50.102210   1.824490           B
3        Ablon-sur-Seine     94001  48.725468   2.419151           B
4  Achères Grand Cormier     78551  48.955183   2.091903           B
📁 Fichier 'gares_sncf_final.csv' créé.


In [9]:
pip install folium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [folium]
Note: you may need to restart the kernel to use updated packages.


In [10]:
import folium
from folium.plugins import MarkerCluster

# 1. Création de la carte centrée sur la France
m = folium.Map(location=[46.603354, 1.888334], zoom_start=6, tiles='OpenStreetMap')

# 2. Création du "Cluster" (le gestionnaire de regroupement de points)
# Cela permet d'éviter que la carte ne soit surchargée et lente
marker_cluster = MarkerCluster().add_to(m)

# 3. Fonction pour choisir la couleur selon l'importance de la gare
# segment_drg : A = Grande gare, B = Moyenne, C = Petite
def get_color(segment):
    if 'A' in str(segment): return 'red'    # Grandes gares (TGV souvent)
    if 'B' in str(segment): return 'orange' # Gares régionales importantes
    return 'blue'                           # Petites gares (le plus fréquent)

print("🌍 Génération de la carte en cours...")

# 4. Boucle pour ajouter chaque gare
# On utilise df_clean.dropna() pour éviter les erreurs s'il manque des coordonnées
for index, row in df_clean.dropna(subset=['latitude', 'longitude']).iterrows():
    
    # On prépare le texte qui s'affiche au clic
    info_bulle = f"""
    <b>{row['nom']}</b><br>
    Code Insee: {row['codeinsee']}<br>
    Type: {row['segment_drg']}
    """
    
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=folium.Popup(info_bulle, max_width=300),
        icon=folium.Icon(color=get_color(row['segment_drg']), icon="train", prefix="fa")
    ).add_to(marker_cluster)

# 5. Sauvegarde
m.save("carte_gares_france.html")
print("✅ Carte générée : ouvrez le fichier 'carte_gares_france.html' dans votre dossier !")

🌍 Génération de la carte en cours...
✅ Carte générée : ouvrez le fichier 'carte_gares_france.html' dans votre dossier !
